In [3]:
import numpy as np
import gmsh
from tqdm import tqdm



# ---------- parameters ----------
lc = 1000.0            # target edge length
layers = 2             # vertical subdivision
stride_x, stride_y = 10, 10
msh_name = "AH_extruded_tetmesh.msh"
# --------------------------------

def load_surface(fname, sx=10, sy=10):
    d = np.genfromtxt(fname, delimiter=",")
    d = d[d[:, 2] != -9999]
    x = np.unique(np.round(d[:, 0], 3))[::sx]
    y = np.unique(np.round(d[:, 1], 3))[::sy]
    keepx, keepy = set(x), set(y)
    return np.array([r for r in d
                     if np.round(r[0], 3) in keepx
                     and np.round(r[1], 3) in keepy])

top = load_surface("../Meshes/3d_meshing/AH_surf_xyz.dat", stride_x, stride_y)
bot = load_surface("../Meshes/3d_meshing/AH_bed_xyz.dat", stride_x, stride_y)

x_vals = np.unique(np.round(top[:, 0], 3))
y_vals = np.unique(np.round(top[:, 1], 3))
nx, ny = len(x_vals), len(y_vals)
x_id = {x: i for i, x in enumerate(x_vals)}
y_id = {y: j for j, y in enumerate(y_vals)}

top_z = np.full((ny, nx), np.nan)
bot_z = np.full((ny, nx), np.nan)
for x, y, z in top: top_z[y_id[y], x_id[x]] = z
for x, y, z in bot: bot_z[y_id[y], x_id[x]] = z

gmsh.initialize()
gmsh.option.setNumber("Mesh.RecombineAll", 0)          # no hex/prisms
gmsh.option.setNumber("Mesh.Algorithm3D", 1)           # Delaunay tet
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)      # Firedrake‑friendly
gmsh.model.add("ExtrudedTetMesh")

pt = {}   # (i,j,level) -> tag

print("Create bottom points …")
for j in tqdm(range(ny)):
    for i in range(nx):
        z = bot_z[j, i]
        if np.isnan(z): continue
        pt[(i, j, 0)] = gmsh.model.geo.addPoint(
            x_vals[i], y_vals[j], z, lc)

# build quads as PlaneSurfaces (no recombination)
print("Create surface quads …")
surfaces = []
for j in range(ny - 1):
    for i in range(nx - 1):
        corners = [(i, j, 0), (i+1, j, 0),
                   (i+1, j+1, 0), (i, j+1, 0)]
        if not all(k in pt for k in corners): continue
        p = [pt[k] for k in corners]
        l = [gmsh.model.geo.addLine(p[k], p[(k+1)%4]) for k in range(4)]
        s = gmsh.model.geo.addPlaneSurface(
            [gmsh.model.geo.addCurveLoop(l)])
        surfaces.append((s, corners))

# extrude each quad into a 3‑D region
print("Extrude and collect volumes …")
vol_tags = []
for s, corners in tqdm(surfaces):
    # height per vertex → we’ll move top nodes later
    dz = [top_z[c[1], c[0]] - bot_z[c[1], c[0]] for c in corners]
    h = float(np.mean(dz))   # use mean for extrusion
    out = gmsh.model.geo.extrude([(2, s)],
                                 0, 0, h,
                                 numElements=[layers],
                                 recombine=False)  # leave as tet
    vol_tags.append(out[1][1])   # volume tag is second return, second elt

gmsh.model.geo.synchronize()

# optional: snap the top layer nodes to the exact top‐surface z
# (skipped for brevity; Firedrake usually tolerates small dz variation)

gmsh.model.mesh.generate(3)
gmsh.write(msh_name)
gmsh.finalize()
print(f"Saved tetra mesh → {msh_name}")

Create bottom points …


100%|██████████| 21/21 [00:00<00:00, 11359.35it/s]


Create surface quads …
Extrude and collect volumes …


100%|██████████| 430/430 [00:09<00:00, 46.75it/s]


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 25 (Line)
Info    : [ 10%] Meshing curve 26 (Line)
Info    : [ 10%] Meshing curve 27 (Line)
Info    : [ 10%] Meshing curve 29 (Line)
Info    : [ 10%] Meshing curve 30 (Line)


In [7]:
#!/usr/bin/env python3
# Smooth extruded tetra mesh — bed & surface from DEM xyz files

import numpy as np
import gmsh
import os
from scipy.interpolate import LinearNDInterpolator
from tqdm import tqdm

# ---------------- user‑set parameters ----------------
lc          = 1000.0          # target edge length in plan view
nz          = 10              # vertical layers
stride_x    = 5               # < 10 for finer plan‑view resolution
stride_y    = 5
msh_name    = "AH_extruded_tetmesh_smooth.msh"
surf_xyz    = "../Meshes/3d_meshing/AH_surf_xyz.dat"
bed_xyz     = "../Meshes/3d_meshing/AH_bed_xyz.dat"
# -----------------------------------------------------

# ------------------------------------------------------------------ helpers
def load_surface(fname, sx=1, sy=1):
    """Return xyz points, subsampled on a stride to keep things light."""
    d = np.genfromtxt(fname, delimiter=",")
    d = d[d[:, 2] > -9990]                 # remove ‑9999 nodata lines
    xs = np.unique(np.round(d[:, 0], 3))[::sx]
    ys = np.unique(np.round(d[:, 1], 3))[::sy]
    keepx, keepy = set(xs), set(ys)
    return np.array([r for r in d
                     if np.round(r[0], 3) in keepx
                     and np.round(r[1], 3) in keepy])

print("Loading DEM xyz …")
top = load_surface(surf_xyz, stride_x, stride_y)
bot = load_surface(bed_xyz,  stride_x, stride_y)

x_vals = np.unique(np.round(top[:, 0], 3))
y_vals = np.unique(np.round(top[:, 1], 3))
nx, ny = len(x_vals), len(y_vals)
x_id = {x: i for i, x in enumerate(x_vals)}
y_id = {y: j for j, y in enumerate(y_vals)}

# gridded elevations for fast lookup
top_z = np.full((ny, nx), np.nan)
bot_z = np.full((ny, nx), np.nan)
for x, y, z in top: top_z[y_id[y], x_id[x]] = z
for x, y, z in bot: bot_z[y_id[y], x_id[x]] = z

# For arbitrary (x,y) lookup later
print("Building scipy griddata interpolators …")
top_interp = LinearNDInterpolator(points=top[:, :2], values=top[:, 2])
bot_interp = LinearNDInterpolator(points=bot[:, :2], values=bot[:, 2])

# ---------------------------------------------------------------- GMSH
gmsh.initialize()
gmsh.option.setNumber("Mesh.RecombineAll", 0)        # no prisms/hex
gmsh.option.setNumber("Mesh.Algorithm3D", 1)         # Delaunay tet
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)    # Firedrake‑friendly
gmsh.model.add("SmoothExtrudedMesh")

pt = {}     # (i, j) → point tag on bed
print("Create bed points …")
for j in tqdm(range(ny)):
    for i in range(nx):
        z = bot_z[j, i]
        if np.isnan(z): continue
        pt[(i, j)] = gmsh.model.geo.addPoint(
            x_vals[i], y_vals[j], z, lc)

# Build base triangles (two per grid square)
print("Create bed triangles …")
planes = []
for j in range(ny - 1):
    for i in range(nx - 1):
        # quad corners
        corners = [(i, j), (i+1, j), (i+1, j+1), (i, j+1)]
        if not all(k in pt for k in corners): continue
        p = [pt[k] for k in corners]
        # lower‑left triangle (p0, p1, p2)
        l1 = [gmsh.model.geo.addLine(p[0], p[1]),
              gmsh.model.geo.addLine(p[1], p[2]),
              gmsh.model.geo.addLine(p[2], p[0])]
        cl1 = gmsh.model.geo.addCurveLoop(l1)
        s1  = gmsh.model.geo.addPlaneSurface([cl1])
        planes.append(s1)
        # upper‑right triangle (p0, p2, p3)
        l2 = [gmsh.model.geo.addLine(p[0], p[2]),
              gmsh.model.geo.addLine(p[2], p[3]),
              gmsh.model.geo.addLine(p[3], p[0])]
        cl2 = gmsh.model.geo.addCurveLoop(l2)
        s2  = gmsh.model.geo.addPlaneSurface([cl2])
        planes.append(s2)

# Extrude every triangle by 1 m; we’ll stretch later
print("Extrude triangles into prisms …")
vols = []
for s in tqdm(planes):
    out = gmsh.model.geo.extrude([(2, s)], 0, 0, 1.0,
                                 numElements=[nz], recombine=False)
    vols.append(out[1][1])          # volume tag

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(3)

# ---------------------------------------------------------- WARP NODES
print("Warping nodes to exact bed & surface elevations …")
node_tags, node_coords, _ = gmsh.model.mesh.getNodes()
coords = node_coords.reshape(-1, 3)

# Pre‑compute column height at each (x, y)
def col_values(x, y):
    zs = top_interp((x, y))
    zb = bot_interp((x, y))
    return zb, zs

# Mesh was extruded 0…1 in z; scale and shift each node
for k, (x, y, z) in enumerate(coords):
    zb, zs = col_values(x, y)
    if np.isnan(zb) or np.isnan(zs):
        continue
    coords[k, 2] = zb + z * (zs - zb)

for tag, (x, y, z) in zip(node_tags, coords):
    gmsh.model.mesh.setNode(tag, [x, y, z], [])


gmsh.write(msh_name)
gmsh.finalize()
print(f"✓  Smooth tetra mesh saved → {msh_name}")


Loading DEM xyz …


Building scipy griddata interpolators …
Create bed points …


100%|██████████| 42/42 [00:00<00:00, 5931.14it/s]


Create bed triangles …
Extrude triangles into prisms …


100%|██████████| 3660/3660 [08:09<00:00,  7.47it/s]


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 25 (Line)
Info    : [ 10%] Meshing curve 26 (Line)
Info    : [ 10%] Meshing curve 27 (Line)
Info    : [ 10%] Meshing curve 29 (Line)
Info    : [ 10%] Meshing curve 31 (Line)
Info    : [ 10%] Meshing curve 32 (Line)
Info    : [ 10%] Meshing curve 33 (Line)
